# Particle diffusion in synthetic turbulent field

In [3]:
from pylab import *

In [ ]:
from crpropa import *

## 1. Generation of turbulent and uniform fields

In [ ]:
# INPUT
B0 = 1 * muG
spacing = 0.01 * pc
grid_number = 1024
index = 5./3.

# TURBULENT FIELD
random_seed = 42
Brms = eta * B0
lmin = 2 * spacing
lmax = (grid_number * spacing) / 8
spectrum = SimpleTurbulenceSpectrum(Brms, lmin, lmax, index)
grid = GridProperties(Vector3d(0), grid_number, spacing)
BField_turbulent = SimpleGridTurbulence(spectrum, grid, random_seed)

# UNIFORM FIELD
ConstMagVec = Vector3d(0 * muG, 0 * muG, B0)
BField_uniform = UniformMagneticField(ConstMagVec)

# TOTAL FIELD
BField = MagneticFieldList()
BField.addField(BField_uniform)
BField.addField(BField_turbulent)

## 2. Particle propagation

In [ ]:
N_particles = 1e3

vec1 = np.round(np.arange(-3.0, 0.2, 0.2), 2)
variable_1 = sys.argv[1]
variable_1 = int(variable_1)
logE = vec1[variable_1 - 1]

vec2 = np.round(np.array((0.5, 1.0, 2.0)), 2)
variable_2 = sys.argv[2]
variable_2 = int(variable_2)
eta = vec2[variable_2 - 1]

# DETERMINATION OF PROPAGATION DISTANCE
E = 10**logE
B = np.sqrt((1e-6**2) + (eta * 1e-6**2)) #G
Z = 1
c = 3e8 # in m/s
time_plateau = 5e7 / E # in yrs
gyration_time = t = ( (E * 1e15) / (Z * B * c**2) ) * 3.17098e-8 # in yrs
gyrations = time_plateau / gyration_time
r_L = ( (E * 1e15) / (Z * B * c) ) * 3.24078e-17 * 1e-6 # in Mpc
len_needed = np.ceil(gyrations * r_L)
len_taken = len_needed + 1

# PROPAGATION
maxlen = len_needed * Mpc
N = 100
step = maxlen / N
sim = ModuleList()
sim.add(PropagationCK(BField))
sim.add(MaximumTrajectoryLength(len_taken * Mpc))

# SOURCE
source = Source()
source.add(SourcePosition(Vector3d(0.)))
source.add(SourceParticleType(nucleusId(1, 1)))
source.add(SourceEnergy(E * PeV))
source.add(SourceIsotropicEmission())
print(source)

# OUPUT
out = TextOutput('trajectory_10tothe{E}_PeV_eta{turb}.txt'.format(E = logE, turb = eta), Output.Trajectory3D)
out.enable(Output.TrajectoryLengthColumn)
out.enable(Output.CurrentPositionColumn)
out.enable(Output.SerialNumberColumn)
out.enable(Output.SourceDirectionColumn)
out.setLengthScale(meter)

# OBSERVER
obs = Observer()
obs.add(ObserverTimeEvolution(step, step, N))
obs.setDeactivateOnDetection(False)
obs.onDetection(out)
sim.add(obs)

# RUN
sim.setShowProgress(True)
sim.run(source, int(N_particles), True)
out.close()

## 3. Computation of diffusion coefficients as function of time

In [ ]:
# LOAD DATA
N_particles = int(1e3)
N_steps = int(1e2+1)#int(1e4+1)
data = np.genfromtxt('Proton/trajectory_10tothe3.0_PeV_eta0.5.txt')

time = data[:,0] / 3e8
serial_number = data[:,1]
x = data[:,4]
y = data[:,5]
z = data[:,6]

t_mat = np.zeros((N_steps, N_particles))
x_mat = np.zeros((N_steps, N_particles))
y_mat = np.zeros((N_steps, N_particles))
z_mat = np.zeros((N_steps, N_particles))
for i in range(1, N_particles+1):
    index_particles = np.where(np.logical_and(serial_number == i, serial_number == i))
    t_mat[:,i-1] = time[index_particles]
    x_mat[:,i-1] = x[index_particles]
    y_mat[:,i-1] = y[index_particles]
    z_mat[:,i-1] = z[index_particles]

# METHOD: dx taken as difference from the origin
x2_mat = x_mat**2
y2_mat = y_mat**2
z2_mat = z_mat**2
xy_mat = np.abs(x_mat * y_mat)

dx2 = np.zeros(N_steps)
dy2 = np.zeros(N_steps)
dz2 = np.zeros(N_steps)
dxy = np.zeros(N_steps)
for j in range(0,N_particles):
    dx2 += x2_mat[:,j]
    dy2 += y2_mat[:,j]
    dz2 += z2_mat[:,j]
    dxy += xy_mat[:,j]
dx2 = dx2 / N_particles
dy2 = dy2 / N_particles
dz2 = dz2 / N_particles
dxy = dxy / N_particles

# DIFFUSION COEFFICIENTS
Dxx = dx2 / (2 * t_mat[:,0])
Dyy = dy2 / (2 * t_mat[:,0])
Dzz = dz2 / (2 * t_mat[:,0])
Dxy = dxy / (2 * t_mat[:,0])
#D = Dxx + Dyy + Dzz

tau = t_mat[:,0] * 3.171e-8 # Time scale in years

plt.figure(figsize=(10, 7))
plt.plot(tau, Dxx, color='blue', label=r'$D_{xx}$')
plt.plot(tau, Dyy, color='red', label=r'$D_{yy}$')
plt.plot(tau, Dzz, color='green', label=r'$D_{zz}$')
plt.plot(tau, Dxy, color='orange', label=r'$D_{xy}$')
#plt.plot(tau, D, color='black', label=r'$D$')
plt.loglog()
plt.xlabel('Time [yrs]', fontsize=17)
plt.ylabel(r'D(E,t) [m$^2$/s]', fontsize=17)
plt.legend(ncol=1, fontsize=17)
plt.title('Diffusion coefficients', fontsize=17)

print('Analysis finished :)')

## 4. Diffusion coefficients as function of $\frac{r_L}{l_c}$

In [ ]:
Z = 1 # protons
eta = 0.5
B_mean = np.sqrt(1e-6**2 + eta*1e-6**2) * 1e-4 # in Tesla
c = 3e8 # in m/s
Lmin = 0.02 * 3.086e+16 # in meters
Lmax = 1.28 * 3.086e+16 # in meters
l_c = 0.2728 * 3.086e+16 # in meters

E_range = np.round(np.arange(-3.0,3.2,0.2),decimals=1)
data_range = len(E_range)
Dxx_E = np.zeros(data_range)
Dyy_E = np.zeros(data_range)
Dzz_E = np.zeros(data_range)
Dxy_E = np.zeros(data_range)

for a in range(0, data_range):

    # LOAD DATA
    N_particles = int(1e3)
    N_steps = int(1e2+1)
    data = np.genfromtxt('trajectory_10tothe{E}_PeV_eta{turb}.txt'.format(E = E_range[a], turb = eta ) )
    time = data[:,0] / 3e8
    if len(time) != N_steps * N_particles:
        print('ERROR: Data size wrong at energy 10^{E}'.format(E = E_range[a]))
        continue
    serial_number = data[:,1]
    x = data[:,4]
    y = data[:,5]
    z = data[:,6]

    t_mat = np.zeros((N_steps, N_particles))
    x_mat = np.zeros((N_steps, N_particles))
    y_mat = np.zeros((N_steps, N_particles))
    z_mat = np.zeros((N_steps, N_particles))
    for i in range(1, N_particles+1):
        index_particles = np.where(np.logical_and(serial_number == i, serial_number == i))
        t_mat[:,i-1] = time[index_particles]
        x_mat[:,i-1] = x[index_particles]
        y_mat[:,i-1] = y[index_particles]
        z_mat[:,i-1] = z[index_particles]

In [ ]:
# METHOD: dx taken as difference from the origin
    x2_mat = x_mat**2
    y2_mat = y_mat**2
    z2_mat = z_mat**2
    xy_mat = np.abs(x_mat * y_mat)

    dx2 = np.zeros(N_steps)
    dy2 = np.zeros(N_steps)
    dz2 = np.zeros(N_steps)
    dxy = np.zeros(N_steps)
    for j in range(0,N_particles):
        dx2 += x2_mat[:,j]
        dy2 += y2_mat[:,j]
        dz2 += z2_mat[:,j]
        dxy += xy_mat[:,j]
    dx2 = dx2 / N_particles
    dy2 = dy2 / N_particles
    dz2 = dz2 / N_particles
    dxy = dxy / N_particles

    # DIFFUSION COEFFICIENTS
    Dxx = dx2 / (2 * t_mat[:,0])
    Dyy = dy2 / (2 * t_mat[:,0])
    Dzz = dz2 / (2 * t_mat[:,0])
    Dxy = dxy / (2 * t_mat[:,0])

    Dxx_E[a] = sum(Dxx[N_steps-15:N_steps]) / 15
    Dyy_E[a] = sum(Dyy[N_steps-15:N_steps]) / 15
    Dzz_E[a] = sum(Dzz[N_steps-15:N_steps]) / 15
    Dxy_E[a] = sum(Dxy[N_steps-15:N_steps]) / 15

# r_L / l_c RATIOS
E = 10**E_range * 1e15
r_L = (E / (Z * B_mean * c)) # in meters
ratio = r_L / l_c

In [ ]:
# PLOT DIFFUSION COEFFICIENTS AS FUNCTION OF (ENERGY) LENGTH SCALRE RATIO
plt.figure(figsize=(10, 7))
plt.plot(ratio, Dxx_E, '-o', color='blue', label=r'$D_{xx}$')
plt.plot(ratio, Dyy_E, '-o', color='red', label=r'$D_{yy}$')
plt.plot(ratio, Dzz_E, '-o', color='green', label=r'$D_{zz}$')
plt.plot(ratio, Dxy_E, '-o', color='orange', label=r'$D_{xy}$')
plt.loglog()
plt.xlabel(r'$r_L / \lambda_c$', fontsize=17)
plt.ylabel(r'$D(E,t)$ [m$^2$/s]', fontsize=17)
plt.legend(ncol=2, fontsize=17)
plt.title(r'Diffusion coefficients of protons in turbulent field with $\eta =$ {turb}'.format(turb = eta), fontsize=17)

# RATIO BETWEEN PERPENDICULAR AND PARALLEL COEFFICIENTS
plt.figure(figsize=(10, 7))
plt.plot(ratio, Dxx_E / Dzz_E, '-o', color='blue', label=r'$D_{xx}/D_{zz}$')
plt.plot(ratio, Dyy_E / Dzz_E, '-o', color='red', label=r'$D_{yy}/D_{zz}$')
plt.loglog()
plt.xlabel(r'$r_L / \lambda_c$', fontsize=17)
plt.ylabel(r'$D_\perp / D_\parallel$', fontsize=17)
plt.legend(ncol=2, fontsize=17)
plt.title(r'Ratio of diffusion coefficients of protons for turbulence $\eta =$ {turb}'.format(turb = eta), fontsize=17)